In [40]:
import numpy as np
import keras
import glob
import os
import random
import matplotlib.pyplot as plt
import datetime
import cv2
%matplotlib inline

In [41]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)  

Mounted at /content/drive


In [42]:
## Generating data

img_width = 100
img_height = 100
img_channel = 3
batch_size = 40

cls = ['cats','dogs']
train_cls_label = {}
train_img_path = []

test_cls_label = {}
test_img_path = []


dataset_path = '/content/drive/My Drive/Colab Notebooks/catdog_dataset/training_set'

for i,cd_class in enumerate(cls):
    path = glob.glob(os.path.join(dataset_path,cd_class,'*'))
    train_img_path = train_img_path + path
    train_cls_label.update({p:i for p in path})

print('train_img_path',len(train_img_path))
print('train_cls_label',len(train_cls_label))
    
dataset_path = '/content/drive/My Drive/Colab Notebooks/catdog_dataset/test_set'
path = []

for i,cd_class in enumerate(cls):
    path = glob.glob(os.path.join(dataset_path,cd_class,'*'))
    test_img_path = test_img_path + path
    test_cls_label.update({p:i for p in path})

print('test_img_path',len(test_img_path))
print('test_cls_label',len(test_cls_label))

train_keys = list(train_cls_label.keys())
test_keys = list(test_cls_label.keys())

random.shuffle(train_keys)
random.shuffle(test_keys)

# To limit 
#train_keys = train_keys #[:4000]
#test_keys = test_keys  #[:2000]

final_train = {}
final_test = {}

final_train.update({p:train_cls_label[p] for p in train_keys})
final_test.update({p:test_cls_label[p] for p in test_keys})



train_img_path 8000
train_cls_label 8000
test_img_path 2000
test_cls_label 2000


In [43]:
print('final_train',len(final_train))
print('final_test',len(final_test))


final_train 8000
final_test 2000


In [44]:
## Define Generator

def generator(dict_cls,batch_size):

  while True:
    
    start_position = 0
    end_position = batch_size

    dict_cls_key = list(dict_cls.keys())

    num_batches = len(dict_cls)//batch_size

    for batch in range(num_batches):
      batch_data = np.zeros((batch_size,img_height,img_width,3))
      batch_label = np.zeros((batch_size,2))

      
      dict_cls_batch_key = dict_cls_key[start_position:end_position]

      for index in range(batch_size):

        image = cv2.imread(dict_cls_batch_key[index])
        image = image[0:image.shape[0]-15,15:image.shape[1]-15]
        image = cv2.resize(image,(img_height,img_width))

        image = image/255
        #print(dict_cls_batch_key[index])
        batch_data[index,:,:,:] = image
        batch_label[index] = keras.utils.to_categorical(dict_cls[dict_cls_batch_key[index]],num_classes=2,dtype='int32')


      start_position = end_position
      end_position = end_position + batch_size
      #print('----------------------------------------------')
      yield batch_data , batch_label



In [45]:
train_generator = generator(final_train,batch_size)
val_generator = generator(final_test,batch_size)

In [46]:
import keras
from keras.models import Sequential , Model
from keras.layers import Dense, Dropout, Activation, Flatten ,BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import optimizers
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

In [47]:
num_classes = 2
num_epochs = 25

In [48]:
model = Sequential()
model.add(Conv2D(32,(3, 3), padding='same',input_shape=(img_height,img_width,3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(32,(3, 3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64,(3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(64,(3, 3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))
#model.add(Dense(64))
#model.add(Activation('relu'))
#model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [49]:
# summary
#opt = optimizers.SGD(learning_rate=0.01,momentum=0.9,nesterov=True)
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['categorical_accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 100, 100, 32)      896       
_________________________________________________________________
activation_21 (Activation)   (None, 100, 100, 32)      0         
_________________________________________________________________
batch_normalization_15 (Batc (None, 100, 100, 32)      128       
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 98, 98, 32)        9248      
_________________________________________________________________
activation_22 (Activation)   (None, 98, 98, 32)        0         
_________________________________________________________________
batch_normalization_16 (Batc (None, 98, 98, 32)        128       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 49, 49, 32)       

In [50]:
if (len(final_train)%batch_size) == 0:
    steps_per_epoch = int(len(final_train)/batch_size)
else:
    steps_per_epoch = (len(final_train)//batch_size) + 1

if (len(final_test)%batch_size) == 0:
    validation_steps = int(len(final_test)/batch_size)
else:
    validation_steps = (len(final_test)//batch_size) + 1

In [51]:
LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00001) # write the REducelronplateau code here

curr_dt_time = datetime.datetime.now()
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
  os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

callbacks_list = [checkpoint,LR]

In [52]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch,epochs=num_epochs,verbose=1, callbacks=callbacks_list,validation_data=val_generator,validation_steps=validation_steps)

Epoch 1/25
200/200 [==============================] - ETA: 0s - loss: 1.1452 - categorical_accuracy: 0.5829
Epoch 00001: val_loss improved from inf to 1.50519, saving model to model_init_2020-08-0607_14_49.980946/model-00001-1.14518-0.58288-1.50519-0.50050.h5
200/200 [==============================] - 77s 387ms/step - loss: 1.1452 - categorical_accuracy: 0.5829 - val_loss: 1.5052 - val_categorical_accuracy: 0.5005
Epoch 2/25
200/200 [==============================] - ETA: 0s - loss: 0.6308 - categorical_accuracy: 0.6441
Epoch 00002: val_loss improved from 1.50519 to 0.66626, saving model to model_init_2020-08-0607_14_49.980946/model-00002-0.63081-0.64412-0.66626-0.61350.h5
200/200 [==============================] - 75s 376ms/step - loss: 0.6308 - categorical_accuracy: 0.6441 - val_loss: 0.6663 - val_categorical_accuracy: 0.6135
Epoch 3/25
200/200 [==============================] - ETA: 0s - loss: 0.5940 - categorical_accuracy: 0.6825
Epoch 00003: val_loss improved from 0.66626 to 0.613